# Preparando ambiente para PySpark

#### Dependências para Python 3.10

In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

#### Configuração das Variáveis de Ambiente

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

#### Tornar PySpark Importável

In [ ]:
# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.4.0-bin-hadoop3')

# Iniciando Ambiente

In [ ]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

#### Carregando dados

In [ ]:
# Faça Upload do seu Arquivo
#from google.colab import files
#f = files.upload()

In [ ]:
import pandas as pd

In [ ]:
#df_pd = pd.read_excel('Data.xlsx')
#Carregando dados
#df_spark = sc.read.csv("train.csv", inferSchema=True, header=True)
#df_sp = sc.createDataFrame(df_pd)

In [ ]:
df = pd.DataFrame.from_dict({
    'partitions': [
        'ano=2023/mes=04/dia=24/sigla="EY4"',
        'ano=2024/mes=04/dia=23/sigla="EY4"',
        'ano=2024/mes=04/dia=22/sigla="EY4"',
        'ano=2024/mes=04/dia=21/sigla="EY4"',
        'ano=2024/mes=04/dia=20/sigla="EY4"',
        'ano=2024/mes=04/dia=19/sigla="EY4"',
        'ano=2024/mes=04/dia=18/sigla="EY4"',
        'ano=2024/mes=04/dia=17/sigla="EY4"',
        'ano=2024/mes=04/dia=16/sigla="EY4"',
        'ano=2024/mes=03/dia=15/sigla="EY4"',
        'ano=2024/mes=03/dia=05/sigla="EY4"',
        'ano=2024/mes=03/dia=04/sigla="EY4"',
        'ano=2024/mes=03/dia=03/sigla="EY4"',
        'ano=2024/mes=03/dia=02/sigla="EY4"',
    ]
})

In [ ]:
dfs = sc.createDataFrame(df)

# Funções Basicas

## Schema

In [ ]:
# ver algumas informações sobre os tipos de dados de cada coluna
dfs.printSchema()

root
 |-- partitions: string (nullable = true)



## Mostrar primeiras linhas

In [ ]:
dfs.show(10)

+--------------------+
|          partitions|
+--------------------+
|ano=2023/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=04/d...|
|ano=2024/mes=03/d...|
+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import split
#column_name = split(dfs.partitions, "=").getItem(0)
qnt_partitions = len(dfs.first()[0].split('/'))
columns_names = dfs.first()[0].split('/')
for partition in range(qnt_partitions):
    dfs = dfs.withColumn(columns_names[partition].split('=')[0], split(split(dfs.partitions, '/').getItem(partition),'=').getItem(1))

In [ ]:
dfs.show(1)

+--------------------+----+---+---+-----+
|          partitions| ano|mes|dia|sigla|
+--------------------+----+---+---+-----+
|ano=2023/mes=04/d...|2023| 04| 24|"EY4"|
+--------------------+----+---+---+-----+
only showing top 1 row



In [ ]:
partitions_columns = dfs.columns
dfs = dfs.drop(partitions_columns[0])
dfs

DataFrame[ano: string, mes: string, dia: string, sigla: string]

In [ ]:
# Sort by "salary" in descending order
dfs.orderBy(dfs.columns, ascending=[False for column in dfs.columns]).show()

+----+---+---+-----+
| ano|mes|dia|sigla|
+----+---+---+-----+
|2024| 04| 23|"EY4"|
|2024| 04| 22|"EY4"|
|2024| 04| 21|"EY4"|
|2024| 04| 20|"EY4"|
|2024| 04| 19|"EY4"|
|2024| 04| 18|"EY4"|
|2024| 04| 17|"EY4"|
|2024| 04| 16|"EY4"|
|2024| 03| 15|"EY4"|
|2024| 03| 05|"EY4"|
|2024| 03| 04|"EY4"|
|2024| 03| 03|"EY4"|
|2024| 03| 02|"EY4"|
|2023| 04| 24|"EY4"|
+----+---+---+-----+



In [ ]:
for predicate in dfs.first():
    print(predicate)

2023
04
24
"EY4"


In [ ]:
from datetime import datetime
from pyspark.sql.functions import col, concat, cast, lit, to_date, unix_timestamp, from_unixtime

In [ ]:

partitions = {
    'ano':"%Y",
    'mes':"%m",
    'dia':"%d",
}
#dfs = dfs.withColumn("data_concatenada", to_date(concat(dfs.ano, lit("-"), dfs.mes, lit("-"), dfs.dia),'yyyy-MM-dd'))
# concat(dfs.ano, lit("-"), dfs.mes, lit("-"), dfs.dia) => 'yyyy-MM-dd'
dfs = dfs.withColumn(
    "data_formatada"
    ,from_unixtime(
        unix_timestamp(
            concat(dfs.ano, dfs.mes, dfs.dia)
            , "yyyyMMdd"
        )
        ,"yyyy-MM-dd"
    )
)
dfs.show(10)

+----+---+---+-----+--------------+
| ano|mes|dia|sigla|data_formatada|
+----+---+---+-----+--------------+
|2023| 04| 24|"EY4"|    2023-04-24|
|2024| 04| 23|"EY4"|    2024-04-23|
|2024| 04| 22|"EY4"|    2024-04-22|
|2024| 04| 21|"EY4"|    2024-04-21|
|2024| 04| 20|"EY4"|    2024-04-20|
|2024| 04| 19|"EY4"|    2024-04-19|
|2024| 04| 18|"EY4"|    2024-04-18|
|2024| 04| 17|"EY4"|    2024-04-17|
|2024| 04| 16|"EY4"|    2024-04-16|
|2024| 03| 15|"EY4"|    2024-03-15|
+----+---+---+-----+--------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import max, character_length

ImportError: cannot import name 'character_length' from 'pyspark.sql.functions' (/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/functions.py)

In [ ]:
mnmax_predicate = dfs.select(max(dfs.data_formatada)).first()[0]
max_predicate.split('-')

['2024', '04', '23']

In [ ]:
from pyspark.sql.functions import character_length

ImportError: cannot import name 'character_length' from 'pyspark.sql.functions' (/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/functions.py)

In [ ]:
ano = 'ano'
a = dfs.select(max(f.length(f.col(ano)))).first()[0]
a

4

In [ ]:
import pyspark.sql.functions as f

In [ ]:
dfs.schema[ano].dataType

StringType()

In [ ]:
d = dfs.printSchema()

root
 |-- ano: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- dia: string (nullable = true)
 |-- sigla: string (nullable = true)
 |-- data_formatada: string (nullable = true)



In [ ]:
d= dfs.schema

In [ ]:
d

StructType([StructField('ano', StringType(), True), StructField('mes', StringType(), True), StructField('dia', StringType(), True), StructField('sigla', StringType(), True), StructField('data_formatada', StringType(), True)])

In [ ]:
for value in d:
    print(value[1])

TypeError: 'StructField' object is not subscriptable

In [ ]:
d = {
    column:dfs.select(
    max(
        f.length(
            f.col(column)
        )
    )
).first()[0] for column in dfs.columns}

In [ ]:
d

{'ano': 4, 'mes': 2, 'dia': 2, 'sigla': 5, 'data_formatada': 10}